In [1]:
import numpy as np
import pandas as pd
from numpy import nan
import pickle
pd.set_option('display.max_rows', 200)

## Data Cleaning

### 1. load data

In [2]:
comprehensive = pd.read_stata('../Data/data/Comprehensive-Sample.dta')

In [16]:
comprehensive.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17920 entries, 0 to 17919
Data columns (total 80 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   year                        17920 non-null  float32
 1   numcode                     17920 non-null  int16  
 2   oilreserves_full            14307 non-null  float32
 3   oilreserves                 12519 non-null  float32
 4   oilreserves_public          9950 non-null   float32
 5   newdiscovery_aspo           12354 non-null  float32
 6   aspo                        4736 non-null   float32
 7   wildcat                     12354 non-null  float32
 8   endowment                   14160 non-null  float32
 9   pop_maddison                10441 non-null  float64
 10  ecgrowth                    9341 non-null   float32
 11  efrac                       390 non-null    float32
 12  lfrac                       390 non-null    float32
 13  rfrac                       390

In [5]:
aspo = pd.read_stata('../Data/data/ASPO-Sample.dta')

In [6]:
aspo.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5120 entries, 0 to 5119
Data columns (total 108 columns):
 #    Column                      Dtype  
---   ------                      -----  
 0    year                        float32
 1    numcode                     int16  
 2    oilreserves_full            float32
 3    oilreserves                 float32
 4    oilreserves_public          float32
 5    newdiscovery_aspo           float32
 6    imputedzero                 float32
 7    aspo                        float32
 8    wildcat                     float32
 9    endowment                   float32
 10   ecgrowth                    float32
 11   country                     object 
 12   incidence2COW               float64
 13   onset2COWCS                 float32
 14   incidenceU                  float32
 15   onsetUCS                    float32
 16   Fearon_war                  float64
 17   onset_FearonCS              float32
 18   Sambanis_war                float64
 19   onse

### 2. function to do transformation

In [8]:
# Create a sample dataframe
df = pd.DataFrame({"country": np.random.choice(["A", "B", "C"], size=20),
                   "year": np.arange(20),
                   "num1": np.random.randint(1, 4, size=20),
                   "num2": np.random.randint(20000, 90000, size=20)})

In [9]:
df = df.sort_values(by=['country','year'], ascending=True)

In [10]:
df

,country,year,num1,num2
0,A,0,1,64271
2,A,2,3,49541
6,A,6,3,70249
7,A,7,2,25423
10,A,10,2,81037
16,A,16,2,59321
17,A,17,1,43499
3,B,3,2,50170
5,B,5,2,28786
8,B,8,3,71639


In [3]:
def transform_df(df, groupby_cols, log_cols, normal_cols):
    
    new_df = df.copy()
    new_cols = []

    for col_name in log_cols:
        new_df[col_name] = np.exp(df[col_name])

    all_cols = log_cols + normal_cols
    
    for col_name in all_cols:
        
        diff_name = col_name + '_diff'
        new_df[diff_name] = new_df.groupby(groupby_cols)[col_name].pct_change()
        new_cols.append(diff_name)
        
    return new_df[new_cols]

In [14]:
transform_df(df, ['country'], ['num1'], ['num2'])

,num1_diff,num2_diff
0,NaN,NaN
2,6.389056,-0.229186
6,0.000000,0.417997
7,-0.632121,-0.638102
10,0.000000,2.187547
16,0.000000,-0.267976
17,-0.632121,-0.266718
3,NaN,NaN
5,0.000000,-0.426231
8,1.718282,1.488675


--

In [31]:
#np.NaN - 5

In [30]:
#comprehensive['code3'].unique()

In [28]:
#comprehensive.query('code3 == "USA"')['logoutreg']
#comprehensive[['year','coup']].groupby('coup').count()

In [29]:
#comprehensive[['year','mountain']].groupby('mountain').count()

### 3. comprehensive - cleaning

In [4]:
v_lst = ['onset2COWCS',
         'onsetUCS',
         'coup',
         'periregular',
         'milexp_pergdpSIPRI',
         #'logmilexgdpSIPRI',
         'numcode',
         'year',
         
         #'logGDP_M',
         'ecgrowth',
         'pop_maddison',
         #'logpop_M',
         'logpopdens', ####
         'democracy',
         'logmountain',
         'ethnic_fractionalization',
         'religion_fractionalization',
         'language_fractionalization',
         'leg_british',
         'no_transition',
         'wildcat',
         
         'valoilres', # value of oil reserves
         #'logvaloilres',
         'valoilres_public',
         #'logvaloilres_public',
         'oilpop',
         #'logoilres',
         'valoilres_impute',
         #'logvaloilres_impute',
         'oilpop_impute',
         #'logoilres_impute',
         'out_regdisaster']
         #'logoutreg'

In [5]:
# variables that need transformation
# transform_lst = ['logmilexgdpSIPRI', # defense burden
#                 'logpop_M',
#                 'logpopdens',
#                 'democracy', # democracy index
#                 'wildcat', # wildcat drilling 
#                 'logoutreg', # out of region disaster 
#                 'logvaloilres',
#                 'logvaloilres_public',
#                 'logoilres',
#                 'logvaloilres_impute',
#                 'logoilres_impute'
#                 ]

transform_lst = ['milexp_pergdpSIPRI',
                'pop_maddison',
                'logpopdens',
                'democracy',
                'wildcat',
                'out_regdisaster',
                'valoilres', # value of oil reserves
                'valoilres_public',
                'oilpop',
                'valoilres_impute',
                'oilpop_impute']

In [6]:
log_lst = ['logpopdens']

normal_lst = ['milexp_pergdpSIPRI',
              'pop_maddison', 
              'democracy',
              'wildcat',
              'out_regdisaster',
              'valoilres', # value of oil reserves
              'valoilres_public', # value of oil reserves from public data
              'oilpop', # oil reserves per capita in million barrels per 1000 persons
              'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
              'oilpop_impute'] # oil reserves per capita - imputed

untransformed_lst = ['onset2COWCS',
                    'onsetUCS',
                    'coup',
                    'periregular',
                    'numcode',
                    'year',
                    'ecgrowth',
                    'logmountain',
                    'ethnic_fractionalization',
                    'religion_fractionalization',
                    'language_fractionalization',
                    'leg_british',
                    'no_transition']

In [7]:
comprehensive = comprehensive.sort_values(by=['numcode','year'], ascending=True)

In [8]:
new_df = transform_df(comprehensive, ['numcode'], 
                     log_lst, 
                     normal_lst)

In [9]:
comprehensive_new = pd.concat([comprehensive[untransformed_lst], new_df], axis = 1)

In [10]:
comprehensive_new.shape

(17920, 24)

In [11]:
comprehensive.to_pickle("../Data/data/comprehensive_new.pkl")  

In [12]:
comprehensive_new

,year,numcode,oilreserves_full,oilreserves,oilreserves_public,newdiscovery_aspo,aspo,wildcat,endowment,pop_maddison,...,logvaloilres_diff,logvaloilres_public_diff,logoilres_impute_diff,logvaloilres_impute_diff,logGDP_M_diff,logpop_M_diff,logpopdens_diff,democracy_diff,logmilexgdpSIPRI_diff,logoutreg_diff
0,1929.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.253965
2,1931.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,1.000000
3,1932.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,1.000000
4,1933.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,3.934184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17915,2004.0,894,0.0,NaN,0.0,NaN,NaN,NaN,0.0,10962.026367,...,NaN,1.0,NaN,NaN,1.005738,1.001604,1.005559,0.000000,NaN,1.035035
17916,2005.0,894,0.0,NaN,0.0,NaN,NaN,NaN,0.0,11115.380859,...,NaN,1.0,NaN,NaN,1.005738,1.001494,1.005156,0.000000,NaN,1.055380
17917,2006.0,894,0.0,NaN,0.0,NaN,NaN,NaN,0.0,11288.252930,...,NaN,1.0,NaN,NaN,1.006773,1.001657,1.005698,0.000000,NaN,0.889851
17918,2007.0,894,0.0,NaN,0.0,NaN,NaN,NaN,0.0,11477.447266,...,NaN,1.0,NaN,NaN,1.006682,1.001781,1.006102,0.000000,NaN,1.035507


In [4]:
np.exp(np.log(3))

3.0000000000000004